<p>What we want: P(Spam | message) = P(Spam | words)</p>
<p>Numerator: P(Spam) * P(words | Spam)</p>
<p>Denominator: P(Spam) * P(words | Spam) + P(not Spam) * P(words | not Spam)</p>
<p>my dictionary : call , free, now, get , bitcoin, send, new</p>

In [1]:
import pandas as pd 
import numpy as np

In [63]:
#remove numeric here 


def remove_numeric(corpus):

    for index, sentence in enumerate(corpus):
        currentSentence = sentence 
        currentSentenceList = currentSentence.split()
        newSentence = filter(lambda x: x.isalpha(), currentSentenceList)
        joinedNewSentence = " ".join(newSentence)
        corpus[index] = joinedNewSentence

In [7]:
spamCollection =  pd.read_csv('Assets/SMSSpamCollection', sep = '\t' , header = None, names=['Label', 'SMS'])
spamCollectionRandom = spamCollection.sample(frac = 1, random_state= 1).reset_index(drop = True)

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
#lets make our dictionary of all words first 

corpus = spamCollection.SMS.reset_index(drop = True)
corpus.head()
corpus = list(corpus)

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though",
 "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv",
 'Even my brother is not like to speak with me. They treat me like aids patent.',
 "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune",
 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
 'Had you

In [69]:
#removing numeric 

remove_numeric(corpus)

In [70]:
corpus

['Go until jurong Available only in bugis n great world la e Cine there got amore',
 'Ok Joking wif u',
 'Free entry in a wkly comp to win FA Cup final tkts May Text FA to to receive entry txt apply',
 'U dun say so early U c already then',
 'Nah I think he goes to he lives around here though',
 'FreeMsg Hey there darling been now and no word like some fun you up for it Tb XxX std chgs to to rcv',
 'Even my brother is not like to speak with They treat me like aids',
 'As per your request Melle Minnaminunginte Nurungu has been set as your callertune for all Press to copy your friends Callertune',
 'As a valued network customer you have been selected to receivea prize To claim call Claim code Valid hours',
 'Had your mobile months or U R entitled to Update to the latest colour mobiles with camera for Call The Mobile Update Co FREE on',
 'gonna be home soon and i want to talk about this stuff anymore cried enough',
 'SIX chances to win From to pounds and send to Cost TsandCs apply Reply H

In [71]:
vectorizer = CountVectorizer(stop_words = ["all", "in", "the", "is" "and", "I", "you", "this", "there"])
matrix = vectorizer.fit_transform(corpus)
result = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names())
result

,aa,aah,aaniye,aaooooright,abdomen,abi,ability,abiola,abj,able,...,yup,zac,zebra,zed,zhong,zindgi,zoe,zoom,zouk,zyada
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# Looks like there are 5572 messages, 6161 words. 

#Let's calculate the sum of each word appearing 
result.loc['Total_Frequency'] = result.sum(axis = 0)

In [74]:
result.tail()

,aa,aah,aaniye,aaooooright,abdomen,abi,ability,abiola,abj,able,...,yup,zac,zebra,zed,zhong,zindgi,zoe,zoom,zouk,zyada
5568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Total_Frequency,1,1,1,1,1,1,2,9,2,26,...,31,1,1,5,1,1,2,1,1,1


In [75]:
result.columns

Index(['aa', 'aah', 'aaniye', 'aaooooright', 'abdomen', 'abi', 'ability',
       'abiola', 'abj', 'able',
       ...
       'yup', 'zac', 'zebra', 'zed', 'zhong', 'zindgi', 'zoe', 'zoom', 'zouk',
       'zyada'],
      dtype='object', length=6161)

In [76]:
dictionaryOfAllWords = {}

for word in result.columns:
    dictionaryOfAllWords[word] = -10

dictionaryOfAllWords


{'aa': -10,
 'aah': -10,
 'aaniye': -10,
 'aaooooright': -10,
 'abdomen': -10,
 'abi': -10,
 'ability': -10,
 'abiola': -10,
 'abj': -10,
 'able': -10,
 'about': -10,
 'aboutas': -10,
 'above': -10,
 'abroad': -10,
 'absolutely': -10,
 'abt': -10,
 'abta': -10,
 'abusers': -10,
 'ac': -10,
 'academic': -10,
 'acc': -10,
 'accenture': -10,
 'accept': -10,
 'access': -10,
 'accidant': -10,
 'accident': -10,
 'accidentally': -10,
 'accommodation': -10,
 'accommodationvouchers': -10,
 'accordin': -10,
 'account': -10,
 'accounting': -10,
 'accounts': -10,
 'ache': -10,
 'achieve': -10,
 'acknowledgement': -10,
 'across': -10,
 'act': -10,
 'acted': -10,
 'actin': -10,
 'acting': -10,
 'action': -10,
 'activate': -10,
 'active': -10,
 'activities': -10,
 'actor': -10,
 'actual': -10,
 'actually': -10,
 'ad': -10,
 'adam': -10,
 'add': -10,
 'added': -10,
 'addicted': -10,
 'addie': -10,
 'adding': -10,
 'address': -10,
 'adds': -10,
 'adewale': -10,
 'adi': -10,
 'admin': -10,
 'administrat

In [77]:
#splitting into training , validation and test sets 
training_test_index = round(len(spamCollectionRandom) * 0.8)
rest_index = len(spamCollectionRandom) - training_test_index

training_set = spamCollectionRandom[:training_test_index].reset_index(drop=True)
validation_set = spamCollectionRandom[training_test_index:training_test_index+ rest_index//2].reset_index(drop=True)
test_set = spamCollectionRandom[training_test_index + rest_index//2:].reset_index(drop=True)

In [78]:
training_set

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
4453,ham,"Sorry, I'll call later in meeting any thing re..."
4454,ham,Babe! I fucking love you too !! You know? Fuck...
4455,spam,U've been selected to stay in 1 of 250 top Bri...
4456,ham,Hello my boytoy ... Geeee I miss you already a...


In [79]:
training_set["Label"].value_counts()


ham     3858
spam     600
Name: Label, dtype: int64

In [80]:
numberOfSpam = 600 
numberOfNotSpam = 3858 
totalNumberOfMessages = len(training_set["Label"])

In [81]:
probabilityOfSpam = numberOfSpam/ totalNumberOfMessages
probabilityOfNotSpam = 1- probabilityOfSpam

<p>Here lets assume that the probability of every word appearing is independent in order for our model to run (Naive Bayes instead of Bayes)</p>
<p>Let's split our train dataset to calculate the probabilities</p>

In [82]:
spams = training_set[training_set["Label"] == "spam"].reset_index(drop= True)
notSpams = training_set[training_set["Label"] == "ham"].reset_index(drop= True)


In [83]:
spams

,Label,SMS
0,spam,FreeMsg Why haven't you replied to my text? I'...
1,spam,Congrats! 2 mobile 3G Videophones R yours. cal...
2,spam,FREE MESSAGE Activate your 500 FREE Text Messa...
3,spam,Call from 08702490080 - tells u 2 call 0906635...
4,spam,Someone has conacted our dating service and en...
...,...,...
595,spam,Congratulations YOU'VE Won. You're a Winner in...
596,spam,Win the newest “Harry Potter and the Order of ...
597,spam,Someone U know has asked our dating service 2 ...
598,spam,YOUR CHANCE TO BE ON A REALITY FANTASY SHOW ca...


In [84]:
notSpams

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
3853,ham,"How about clothes, jewelry, and trips?"
3854,ham,"Sorry, I'll call later in meeting any thing re..."
3855,ham,Babe! I fucking love you too !! You know? Fuck...
3856,ham,Hello my boytoy ... Geeee I miss you already a...


In [44]:
spamsList = list(spams["SMS"])
remove_numeric(spamsList)

<p><b>THIS IS NUMERATOR (SPAM) CALCULATION </b></p>

In [86]:


vectorizerSpam = CountVectorizer()

matrix = vectorizerSpam.fit_transform(spamsList)
resultSpam = pd.DataFrame(matrix.toarray(), columns = vectorizerSpam.get_feature_names())
resultSpam


,about,abroad,abta,ac,access,accident,accommodation,accommodationvouchers,account,action,...,ymca,you,your,yourinclusive,yours,yr,yrs,zebra,zed,zouk
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [136]:
#get index of all of our dictionary 


# dictionary = defaultdict(0)
dictionaryOfKeyWords = {"call": 0 , "free": 0, "now" : 0, "get": 0, "send": 0 , "new" : 0, "bitcoin": -10}
# since bitcoin is not in the dictionary, we apply laplace smoothing to make it a probability of 1/2 

for word in dictionaryOfKeyWords: 
    try:

        dictionaryOfKeyWords[word] = vectorizerSpam.get_feature_names().index(word)
    except ValueError:
        print(word,"is not in the message! Leplace smoothing will be applied")

print(dictionaryOfKeyWords)


bitcoin is not in the message! Leplace smoothing will be applied
{'call': 146, 'free': 426, 'now': 785, 'get': 451, 'send': 1024, 'new': 765, 'bitcoin': -10}


In [137]:
# calculate the probabilty of word given that is a spam message
# P(spam) * P(every single dictionary | spam)

# P(every single dictionary | spam) = P(appearing) vs not appearing
# if a word didnt appear, give the default likelihood to be 1/2 (laplace smoothing)




# Count the probability of every single word appearing in my list of special words
vectorizedMatrix = matrix.toarray()
def probabiltyOfWordInaSpamExcludingOther(dictionary, vectorizedMatrix, probabilityOfSpam):
    res = 1 
    probabilities = {}
    totalNumberOfMessages = len(vectorizedMatrix)
    for word in dictionary:
        if dictionary[word] != -10:
            count = 0
            for array in vectorizedMatrix:
                if array[dictionary[word]] == 1:
                    count += 1
            
            probability = count /totalNumberOfMessages
            probabilities[word] = probability
        else:
            probabilities[word] = 1/2



    
    # then use all the probabilities to calculate the final numerator here 
    for prob in probabilities:
        res *= probabilities[prob]

    #lets say my total vocabulary is 2000 words for simplicity 

    return res * probabilityOfSpam




    



In [138]:
# Count the probability the probability of every other word not appearing 

resultSpam




,about,abroad,abta,ac,access,accident,accommodation,accommodationvouchers,account,action,...,ymca,you,your,yourinclusive,yours,yr,yrs,zebra,zed,zouk
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
resultSpam.loc["Total_Frequency"] = resultSpam.sum(axis = 0)
resultSpam

,about,abroad,abta,ac,access,accident,accommodation,accommodationvouchers,account,action,...,ymca,you,your,yourinclusive,yours,yr,yrs,zebra,zed,zouk
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
TotalFrequency = dictionaryOfAllWords


for column in resultSpam.columns:
    TotalFrequency[column] = resultSpam.iloc[-1][column]
print(type(TotalFrequency["about"]))

<class 'numpy.int64'>


In [141]:
def calculateProbabilityOfWordNotInList(TotalFrequency, dictionaryOfKeyWords, vectorizedMatrix):

    res = 1
    totalNumberOfMessages = len(vectorizedMatrix)
    for word in TotalFrequency:
        frequency = TotalFrequency[word]
        if frequency != -10 and word not in dictionaryOfKeyWords:
            probability = (totalNumberOfMessages - frequency) / totalNumberOfMessages
        elif frequency == -10 and word not in dictionaryOfKeyWords:
            probability = 1/2
            
        res *= probability
            

    return res 

# the numerator value here is: 

probabilityOfWordsNotInList = calculateProbabilityOfWordNotInList(TotalFrequency, dictionaryOfKeyWords, vectorizedMatrix)
numerator = probabilityOfWordsNotInList * probabiltyOfWordInaSpamExcludingOther(dictionaryOfKeyWords, vectorizedMatrix, probabilityOfSpam)

            

<p> Now for words keywords which exist, given that the message is NOT a spam <p>

In [142]:
probabilityOfNotSpam 

0.8654104979811574

In [143]:
#reset the frequency of words 
for word in dictionaryOfAllWords:
    dictionaryOfAllWords[word] = -10



In [144]:
# count frequency of words in NOT SPAM LIST 

notSpamsList = notSpams["SMS"]
notSpamsList

0                                 by the pretty sculpture
1                                Are you going to make me
2                              Welp apparently he retired
3                                                        
4       I forgot ask ü all a card on da present Ü all ...
                              ...                        
3853                                        How about and
3854    call later in meeting any thing related to tra...
3855    I fucking love you too You Fuck it was so good...
3856    Hello my boytoy Geeee I miss you already and I...
3857                                            my boytoy
Name: SMS, Length: 3858, dtype: object

In [145]:
remove_numeric(notSpamsList)

In [146]:
vectorizerNotSpam = CountVectorizer()

matrix = vectorizerNotSpam.fit_transform(spamsList)
resultNotSpam = pd.DataFrame(matrix.toarray(), columns = vectorizerNotSpam.get_feature_names())
resultNotSpam

,about,abroad,abta,ac,access,accident,accommodation,accommodationvouchers,account,action,...,ymca,you,your,yourinclusive,yours,yr,yrs,zebra,zed,zouk
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [147]:
resultNotSpam.loc["Total_Frequency"] = resultNotSpam.sum(axis = 0)
resultNotSpam.tail()


,about,abroad,abta,ac,access,accident,accommodation,accommodationvouchers,account,action,...,ymca,you,your,yourinclusive,yours,yr,yrs,zebra,zed,zouk
596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
598,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Total_Frequency,7,1,1,4,4,2,1,1,13,3,...,1,215,210,1,6,7,2,1,4,1


In [148]:
dictionaryOfKeyWords

{'call': 146,
 'free': 426,
 'now': 785,
 'get': 451,
 'send': 1024,
 'new': 765,
 'bitcoin': -10}

In [149]:
for word in dictionaryOfKeyWords: 
    try:

        dictionaryOfKeyWords[word] = vectorizerNotSpam.get_feature_names().index(word)
    except ValueError:
        print(word,"is not in the message! Leplace smoothing will be applied")

print(dictionaryOfKeyWords)


bitcoin is not in the message! Leplace smoothing will be applied
{'call': 146, 'free': 426, 'now': 785, 'get': 451, 'send': 1024, 'new': 765, 'bitcoin': -10}


In [150]:
probabilityOfKeyWordAppearing = probabiltyOfWordInaSpamExcludingOther(dictionaryOfKeyWords, vectorizedMatrix, probabilityOfNotSpam)
print(probabilityOfKeyWordAppearing)

3.454184557350082e-06


In [151]:
vectorizedMatrix = matrix.toarray()

In [153]:
for column in resultNotSpam.columns:
    TotalFrequency[column] = resultSpam.iloc[-1][column]
    

In [154]:
probabilityOfNotKeyWords = calculateProbabilityOfWordNotInList(TotalFrequency, dictionaryOfKeyWords, vectorizedMatrix)

In [155]:
denominator = probabilityOfKeyWordAppearing * probabilityOfNotKeyWords 

In [156]:
denominator += numerator

In [157]:
print(numerator / denominator)

nan


C:\Users\admin\AppData\Local\Temp/ipykernel_10088/1128423616.py:1: RuntimeWarning: invalid value encountered in double_scalars
  print(numerator / denominator)
